In [17]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db_local/"
#datadir = "/datasets/eurora_data/db/"
#datadir = "C:/Users/folid/git-repo/hpc_job_power_prediction/datasets/eurora_data/db_local/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"
infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"

## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')

infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"
#infile_jobs = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow_t2_54_234192"

#infile_jobs = datadir + "CPUs/jobs_cleaned_whole"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [18]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [19]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [20]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [21]:
#jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
#print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### Read jobs data

drop the jobs out of interval

In [22]:
jobs_whole = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs_whole.shape[0]))

jobs_whole_data contains 336787 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
interval_jobs = jobs_whole[pd.to_datetime(jobs_whole['end_time']) <= end_time]
interval_jobs = interval_jobs[pd.to_datetime(interval_jobs['run_start_time']) > start_time]
print("Alina set contains {} records".format(interval_jobs.shape[0]))

Alina set contains 336787 records


#### merge

In [24]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, interval_jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

381008


In [25]:
### remove to continue from a specific job
##long_jobs = long_jobs.drop(['real_pow', 'runned_alone', 'real_pow_quality'], axis=1) #old?

interval_jobs['real_pow'] = 0.0
interval_jobs['ran_alone'] = True
interval_jobs['real_pow_quality'] = 0.0

interval_jobs['n_2_1'] = 0
interval_jobs['n_2_2'] = 0
interval_jobs['n_3_1'] = 0

interval_jobs['job_tot_timepoints'] = 0
interval_jobs['job_timepoints'] = 0
interval_jobs['good_nodes'] = 0 # TODO add this on the file? maybe I can drop it at the end


In [26]:
import time

start_time = time.time()
i = 0
#i = 234193 # remove to continue from a specific job
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_jobs_real_pow"

for node in nodes:
    # recupero le allocazioni nei vari nodi
    merged_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    print("---------- {} jobs in node {}".format(merged_jobs_to_node.shape[0], node))
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment + "_active_cores_and_jobs.csv"
    node_measurements = pd.read_csv(infile_node)
    node_measurements['timestamp'] = pd.to_datetime(node_measurements['timestamp'])
    node_start_time = pd.to_datetime(node_measurements['timestamp'].iloc[0])
    num_minutes = node_measurements.shape[0]
    print("-------------------node data starts at {}, num minute {}".format(node_start_time, num_minutes))
    
    # per ogni nodo recupero ogni allocazione
    for index_job_to_node, row_job_to_node in merged_jobs_to_node.iterrows():
        #print()
        if(i % 100 == 0):
            print("{}/{} {}".format(i, merged_jobs_to_nodes.shape[0], row_job_to_node['job_id_string']))
        
        partial_pow_consumption = 0
        job_id_string = row_job_to_node['job_id_string']
        
        # recupero indice del job in interval_jobs
        job = interval_jobs[interval_jobs['job_id_string'] == job_id_string]
        job_index = interval_jobs[interval_jobs['job_id_string'] == job_id_string].index
        if(job_index != None):
            # recupero eventuali valori scritti in precedenti iterazioni
            job_real_consumption = job['real_pow'].iloc[0] 
            job_n_2_1 = job['n_2_1'].iloc[0]
            job_n_2_2 = job['n_2_2'].iloc[0]
            job_n_3_1 = job['n_3_1'].iloc[0]
            job_tot_timepoints = job['job_tot_timepoints'].iloc[0]
            job_timepoints = job['job_timepoints'].iloc[0]
            job_good_nodes = job['good_nodes'].iloc[0]
            job_ran_alone = job['ran_alone'].iloc[0]

            node_type = nodes_data.iloc[row_job_to_node['node_id']-1]['cpu_type']
            if(node_type == '2_1_ghz'):
                job_n_2_1 += 1
            elif(node_type == '2_2_ghz'):
                job_n_2_2 += 1
            elif(node_type == '3_1_ghz'):
                job_n_3_1 += 1

            node_idle_core = nodes_data.iloc[row_job_to_node['node_id']-1]['core_idle']
            
            job_start_time = pd.to_datetime(row_job_to_node['run_start_time'])
            job_end_time = pd.to_datetime(row_job_to_node['end_time'])
            
            if(job_end_time - job_start_time >= np.timedelta64(5, 's')):
                before_minutes = int((job_start_time - node_start_time) / np.timedelta64(5, 's'))
                running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
                after_minutes = num_minutes - running_minutes - before_minutes
                
                before_serie = pd.Series(False, index=np.arange(before_minutes))
                running_serie = pd.Series(True, index=np.arange(running_minutes))
                after_serie = pd.Series(False, index=np.arange(after_minutes))
                concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
                
                interval_measurements = node_measurements[concat_series]
                
                ### take only the entries of the interval where the job was running
                #interval_measurements = node_measurements.loc[(pd.to_datetime(node_measurements['timestamp']) >= pd.to_datetime(row_job_to_node['run_start_time'])) & (pd.to_datetime(node_measurements['timestamp']) <= pd.to_datetime((row_job_to_node['end_time'])) - np.timedelta64(5, 's'))]
                ### drop missing measurements (calculate the percentage)
                n_node_measurements = interval_measurements.shape[0]
                job_tot_timepoints += n_node_measurements

                interval_measurements = interval_measurements.dropna()
                #print("{}/{} intervals missing".format(n_node_measurements-interval_measurements.shape[0], n_node_measurements))
                job_timepoints += n_node_measurements-interval_measurements.shape[0]

                ### drop where active cores is greater than 16 - this problem occours only once on node 30
                interval_measurements = interval_measurements[interval_measurements['active_cores'] <= 16]
                ### drop row with 0 active_cores (or less than the current job used cores)
                interval_measurements = interval_measurements[interval_measurements['active_cores'] >= row_job_to_node['ncpus']]  

                if(interval_measurements.shape[0] != 0):
                    job_good_nodes += 1 


                ### group the intervals wehere the partial_pow_cons can be obtaied with the same instance of the formula
                ### take the mean for the pow columns
                # interval_grouped = interval_measurements.reset_index().groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"]).mean()
                interval_grouped = interval_measurements.groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"])
                counts = interval_grouped.size().to_frame(name='counts')
                interval_grouped = (counts
                 .join(interval_grouped.agg({'pow_tot':'mean'}).rename(columns={'pow_tot': 'pow_tot_mean'}))
                 .reset_index()
                )
                interval_grouped.sort_values('active_jobs')

                if(interval_grouped.shape[0]>1):
                    job_ran_alone = False


                if(interval_grouped.shape[0] != 0):
                    #interval_grouped['pow_tot'] = (interval_grouped['pow_tot_0_mean'] + interval_grouped['pow_tot_1_mean']) * row['ncpus'] / interval_grouped['active_cores'] 
                    interval_grouped['pow_tot'] = (interval_grouped['pow_tot_mean'] - (16 - interval_grouped['active_cores'])*node_idle_core) * row_job_to_node['ncpus'] / interval_grouped['active_cores'] 

                    # not_alone_counts = interval_grouped['counts'].loc[interval_grouped['active_jobs'] != 1].sum()
                    # interval_grouped.loc[interval_grouped['active_jobs'] == 1, ['counts']] += not_alone_counts
                    #print(interval_grouped)
                    partial_pow_consumption = np.average(interval_grouped['pow_tot'], weights=interval_grouped['counts'])
                    # partial_pow_consumption = partial_pow_consumption / interval_grouped.shape[0]
                    # print(interval_grouped)

                #print("partial measurement: {}".format(partial_pow_consumption))
                job_real_consumption += partial_pow_consumption
                partial_pow_consumption = 0

        
            else: # jobs shorter than 5 sec
                   # keep the only timepoint
                job_start_time = row_job_to_node['run_start_time']
                #job_end_time = row_job_to_node['end_time']
                if(int(job_start_time[18:]) >= 5):
                    job_start_time = pd.to_datetime(job_start_time[:-1] + "5")
                else:
                    job_start_time = pd.to_datetime(job_start_time[:-1] + "0")
                #if(int(job_end_time[18:]) >= 5):
                #    job_end_time = pd.to_datetime(job_end_time[:-1] + "0") + np.timedelta64(10, 's')
                #else:
                #    job_end_time = pd.to_datetime(job_end_time[:-1] + "5")
                
                measure = node_measurements[node_measurements['timestamp'] == job_start_time]
                job_tot_timepoints += 1
                
                measure = measure.dropna()
                if(measure.shape[0] != 0):
                    
                    job_good_nodes += 1
                    if(measure['active_cores'].iloc[0] != 0):
                        job_ran_alone = False

                    active_cores = measure['active_cores'].iloc[0] + row_job_to_node['ncpus']
                    if(active_cores > 16):
                        active_cores = 16
                    job_real_consumption += (measure['pow_tot'].iloc[0] - (16 - active_cores) * node_idle_core) * row_job_to_node['ncpus'] / active_cores
                else:
                    job_timepoints += 1
                    #print("error measure")
                    #print(row_job_to_node['run_start_time'])
                    job_pow_quality = 0.0
               
            
            if(job_real_consumption < 0):
                job_real_consumption = 0.0
            #print("job_real_consumption: {}".format(job_real_consumption))
            
            interval_jobs.at[job_index, 'real_pow'] = job_real_consumption
            interval_jobs.at[job_index, 'n_2_1'] = job_n_2_1
            interval_jobs.at[job_index, 'n_2_2'] = job_n_2_2
            interval_jobs.at[job_index, 'n_3_1'] = job_n_3_1
            interval_jobs.at[job_index, 'job_tot_timepoints'] = job_tot_timepoints
            interval_jobs.at[job_index, 'job_timepoints'] = job_timepoints
            interval_jobs.at[job_index, 'good_nodes'] = job_good_nodes
            interval_jobs.at[job_index, 'ran_alone'] = job_ran_alone
    
        
        i = i + 1
    #if(i % 10000 == 0):
    interval_jobs.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")
    print("--- %s seconds ---" % (time.time() - start_time))
        
interval_jobs.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")

---------- 1746 jobs in node 01
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
0/381008 499903.node129
100/381008 528713.node129
200/381008 584230.node129
300/381008 590214.node129
400/381008 597118.node129
500/381008 628373.node129
600/381008 813096.node129
700/381008 921942.node129
800/381008 927296.node129
900/381008 937127.node129
1000/381008 1016374.node129
1100/381008 1200805.node129
1200/381008 1207487.node129
1300/381008 1209577.node129
1400/381008 1211302.node129
1500/381008 1286765.node129
1600/381008 1309605.node129
1700/381008 1319540.node129
--- 246.75038290023804 seconds ---
---------- 240 jobs in node 02
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
1800/381008 571561.node129
1900/381008 593319.node129
--- 297.7532322406769 seconds ---
---------- 1625 jobs in node 03
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
2000/381008 502690.node129
2100/381008 510890.node129
2200/3810

20200/381008 583700.node129
20300/381008 635854.node129
20400/381008 891606.node129
20500/381008 927001.node129
20600/381008 1007827.node129
20700/381008 1028206.node129
20800/381008 1199540.node129
20900/381008 1205545.node129
21000/381008 1209887.node129
21100/381008 1286784.node129
--- 2984.426213502884 seconds ---
---------- 1106 jobs in node 19
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
21200/381008 505267.node129
21300/381008 584869.node129
21400/381008 600159.node129
21500/381008 630610.node129
21600/381008 815492.node129
21700/381008 912523.node129
21800/381008 926978.node129
21900/381008 1008414.node129
22000/381008 1016037.node129
22100/381008 1211396.node129
22200/381008 1237831.node129
--- 3134.1780960559845 seconds ---
---------- 1367 jobs in node 20
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
22300/381008 504913.node129
22400/381008 582985.node129
22500/381008 596009.node129
22600/381008 622052.node129

41500/381008 813931.node129
41600/381008 817839.node129
41700/381008 907565.node129
41800/381008 922635.node129
41900/381008 926968.node129
42000/381008 935636.node129
42100/381008 936676.node129
42200/381008 1007073.node129
42300/381008 1013744.node129
42400/381008 1017201.node129
42500/381008 1200365.node129
42600/381008 1207589.node129
42700/381008 1208618.node129
42800/381008 1210050.node129
42900/381008 1211424.node129
43000/381008 1237659.node129
43100/381008 1295979.node129
43200/381008 1302419.node129
43300/381008 1310354.node129
43400/381008 1318190.node129
43500/381008 1319422.node129
43600/381008 1329302.node129
43700/381008 1330499.node129
43800/381008 1331453.node129
43900/381008 1332337.node129
44000/381008 1333047.node129
44100/381008 1333411.node129
44200/381008 1333757.node129
44300/381008 1334071.node129
44400/381008 1334384.node129
44500/381008 1334736.node129
44600/381008 1335022.node129
44700/381008 1335317.node129
44800/381008 1335605.node129
44900/381008 1335875.

68200/381008 586870.node129
68300/381008 589043.node129
68400/381008 589245.node129
68500/381008 593497.node129
68600/381008 596169.node129
68700/381008 598575.node129
68800/381008 600970.node129
68900/381008 607519.node129
69000/381008 623196.node129
69100/381008 625066.node129
69200/381008 631235.node129
69300/381008 636364.node129
69400/381008 800258.node129
69500/381008 804420.node129
69600/381008 811596.node129
69700/381008 811698.node129
69800/381008 813675.node129
69900/381008 824451.node129
70000/381008 907695.node129
70100/381008 908739.node129
70200/381008 922430.node129
70300/381008 925675.node129
70400/381008 927187.node129
70500/381008 936440.node129
70600/381008 1007306.node129
70700/381008 1016151.node129
70800/381008 1198839.node129
70900/381008 1205521.node129
71000/381008 1206635.node129
71100/381008 1210036.node129
71200/381008 1234414.node129
71300/381008 1235801.node129
71400/381008 1235942.node129
71500/381008 1236242.node129
71600/381008 1241662.node129
71700/381

94400/381008 1329352.node129
94500/381008 1413082.node129
94600/381008 1415741.node129
--- 12123.30496096611 seconds ---
---------- 26513 jobs in node 44
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
94700/381008 500321.node129
94800/381008 504774.node129
94900/381008 509185.node129
95000/381008 511838.node129
95100/381008 520272.node129
95200/381008 520375.node129
95300/381008 520477.node129
95400/381008 520588.node129
95500/381008 520694.node129
95600/381008 520800.node129
95700/381008 520981.node129
95800/381008 521180.node129
95900/381008 521386.node129
96000/381008 521590.node129
96100/381008 521790.node129
96200/381008 521995.node129
96300/381008 522197.node129
96400/381008 522409.node129
96500/381008 522616.node129
96600/381008 522830.node129
96700/381008 523031.node129
96800/381008 523231.node129
96900/381008 523432.node129
97000/381008 523631.node129
97100/381008 523833.node129
97200/381008 524033.node129
97300/381008 524247.node129
97400/38100

...

256300/381008 846367.node129
256400/381008 846668.node129
256500/381008 846969.node129
256600/381008 847275.node129
256700/381008 847580.node129
256800/381008 847881.node129
256900/381008 848187.node129
257000/381008 848490.node129
257100/381008 848793.node129
257200/381008 849091.node129
257300/381008 849400.node129
257400/381008 849699.node129
257500/381008 849998.node129
257600/381008 850302.node129
257700/381008 850600.node129
257800/381008 850903.node129
257900/381008 851203.node129
258000/381008 851504.node129
258100/381008 851806.node129
258200/381008 852104.node129
258300/381008 852407.node129
258400/381008 852706.node129
258500/381008 853007.node129
258600/381008 853311.node129
258700/381008 853617.node129
258800/381008 853917.node129
258900/381008 854220.node129
259000/381008 854523.node129
259100/381008 854827.node129
259200/381008 855130.node129
259300/381008 855433.node129
259400/381008 855732.node129
259500/381008 856034.node129
259600/381008 856333.node129
259700/381008 

283600/381008 851894.node129
283700/381008 852195.node129
283800/381008 852498.node129
283900/381008 852798.node129
284000/381008 853102.node129
284100/381008 853402.node129
284200/381008 853708.node129
284300/381008 854008.node129
284400/381008 854311.node129
284500/381008 854611.node129
284600/381008 854915.node129
284700/381008 855218.node129
284800/381008 855520.node129
284900/381008 855820.node129
285000/381008 856124.node129
285100/381008 856427.node129
285200/381008 856794.node129
285300/381008 857205.node129
285400/381008 880522.node129
285500/381008 888709.node129
285600/381008 937550.node129
285700/381008 939203.node129
285800/381008 939421.node129
285900/381008 939622.node129
286000/381008 939827.node129
286100/381008 940036.node129
286200/381008 940241.node129
286300/381008 940447.node129
286400/381008 940738.node129
286500/381008 941146.node129
286600/381008 941551.node129
286700/381008 949283.node129
286800/381008 953626.node129
286900/381008 954022.node129
287000/381008 

310600/381008 1233890.node129
310700/381008 1242944.node129
310800/381008 1243350.node129
310900/381008 1243684.node129
311000/381008 1244148.node129
311100/381008 1296547.node129
311200/381008 1319344.node129
311300/381008 1414371.node129
--- 37647.59327340126 seconds ---
---------- 20813 jobs in node 59
-------------------node data starts at 2014-03-31 00:00:00, num minute 3715201
311400/381008 502841.node129
311500/381008 503960.node129
311600/381008 510116.node129
311700/381008 565472.node129
311800/381008 581176.node129
311900/381008 582843.node129
312000/381008 590715.node129
312100/381008 596713.node129
312200/381008 599509.node129
312300/381008 600770.node129
312400/381008 600895.node129
312500/381008 601352.node129
312600/381008 601478.node129
312700/381008 601583.node129
312800/381008 601686.node129
312900/381008 601914.node129
313000/381008 602018.node129
313100/381008 602654.node129
313200/381008 603608.node129
313300/381008 605226.node129
313400/381008 605332.node129
31350

337800/381008 575025.node129
337900/381008 575126.node129
338000/381008 575234.node129
338100/381008 583374.node129
338200/381008 589562.node129
338300/381008 595774.node129
338400/381008 601037.node129
338500/381008 622820.node129
338600/381008 805139.node129
338700/381008 923523.node129
338800/381008 935793.node129
338900/381008 938472.node129
339000/381008 1013768.node129
339100/381008 1198828.node129
339200/381008 1200347.node129
339300/381008 1201513.node129
339400/381008 1207429.node129
339500/381008 1209632.node129
339600/381008 1210952.node129
339700/381008 1218198.node129
339800/381008 1218350.node129
339900/381008 1218516.node129
340000/381008 1228334.node129
340100/381008 1230416.node129
340200/381008 1234659.node129
340300/381008 1239123.node129
340400/381008 1244442.node129
340500/381008 1298113.node129
340600/381008 1316357.node129
340700/381008 1319537.node129
340800/381008 1320329.node129
340900/381008 1321016.node129
341000/381008 1321735.node129
341100/381008 1322434.

363500/381008 832753.node129
363600/381008 832855.node129
363700/381008 832956.node129
363800/381008 833059.node129
363900/381008 833170.node129
364000/381008 833271.node129
364100/381008 833374.node129
364200/381008 833486.node129
364300/381008 833713.node129
364400/381008 833947.node129
364500/381008 834208.node129
364600/381008 834556.node129
364700/381008 841557.node129
364800/381008 841861.node129
364900/381008 842167.node129
365000/381008 842470.node129
365100/381008 842771.node129
365200/381008 843079.node129
365300/381008 843384.node129
365400/381008 843688.node129
365500/381008 843990.node129
365600/381008 844298.node129
365700/381008 844606.node129
365800/381008 844907.node129
365900/381008 845212.node129
366000/381008 845517.node129
366100/381008 845820.node129
366200/381008 846132.node129
366300/381008 846435.node129
366400/381008 846736.node129
366500/381008 847037.node129
366600/381008 847345.node129
366700/381008 847649.node129
366800/381008 847952.node129
366900/381008 

In [36]:
#job_index = interval_jobs[(interval_jobs['real_pow'] != 0.0) & (interval_jobs['job_timepoints'] == 0)].index


In [38]:
#interval_jobs.at[job_index, 'job_timepoints'] = 1


In [40]:
interval_jobs['real_pow_quality'] = 1 - (interval_jobs['job_timepoints'] / interval_jobs['job_tot_timepoints'])

In [41]:
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_jobs_real_pow"
interval_jobs.to_csv(outfile + ".csv")

In [ ]:
#debug

In [2]:
infile_jobs = datadir + "CPUs/" + interval_comment + "/Alina_jobs_real_pow"
jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_data contains {} records".format(jobs.shape[0]))

jobs_data contains 336787 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
jobs['real_pow_quality'].mean()

0.5942813919519043

In [4]:
#job_end_time - job_start_time >= np.timedelta64(5, 's')


#job_index_0 = jobs[(jobs['job_tot_timepoints'] == 1) & (jobs['job_timepoints'] == 0)].index
#job_index_1 = jobs[(jobs['job_tot_timepoints'] == 1) & (jobs['job_timepoints'] == 1)].index

In [6]:
short_jobs = jobs[(pd.to_datetime(jobs['end_time']) - pd.to_datetime(jobs['run_start_time']) < np.timedelta64(5, 's'))]
long_jobs = jobs[(pd.to_datetime(jobs['end_time']) - pd.to_datetime(jobs['run_start_time']) >= np.timedelta64(5, 's'))]

In [8]:
print("{} {}".format(jobs.shape[0], short_jobs.shape[0] + long_jobs.shape[0]))

336787 336787


In [11]:
short_jobs[['job_timepoints', 'job_tot_timepoints']]

,job_timepoints,job_tot_timepoints
job_id,,
205795,1,1
205796,1,1
205797,1,1
205798,1,1
205799,1,1
205802,1,1
205803,1,1
205804,1,1
205805,1,1


In [12]:
short_jobs['job_timepoints'] = short_jobs['job_tot_timepoints'] - short_jobs['job_timepoints']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
short_jobs[['job_timepoints', 'job_tot_timepoints']]

,job_timepoints,job_tot_timepoints
job_id,,
205795,0,1
205796,0,1
205797,0,1
205798,0,1
205799,0,1
205802,0,1
205803,0,1
205804,0,1
205805,0,1


In [14]:
short_jobs['job_timepoints'].sum()

17215

In [15]:
short_jobs.shape[0]

145098

In [16]:
jobs_2 = pd.concat([short_jobs, long_jobs])

In [17]:
jobs['real_pow_quality'] = 1.0 - (jobs['job_timepoints'] / jobs['job_tot_timepoints'])
jobs['real_pow_quality'].mean()

0.5942813919519043

In [18]:
jobs_2['real_pow_quality'] = 1.0 - (jobs_2['job_timepoints'] / jobs_2['job_tot_timepoints'])
jobs_2['real_pow_quality'].mean()

0.9237523929109674

In [19]:
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_jobs_real_pow"
jobs_2.to_csv(outfile + ".csv")

In [10]:
infile_jobs = datadir + "CPUs/" + interval_comment + "/Alina_long_jobs_real_pow"
jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_data contains {} records".format(jobs.shape[0]))

jobs_data contains 336787 records


In [11]:
jobs['real_pow_quality'].mean()

0.45639819188001324

In [12]:
jobs['real_pow_quality_2'] = 1.0 - (jobs['job_timepoints'] / jobs['job_tot_timepoints'])

In [13]:
jobs['real_pow_quality_2'].mean()

0.5436018081199867